In [ ]:
!pip install EMD-signal

In [13]:
import pickle
import numpy as np
import pandas as pd
from PyEMD import EMD
import tensorflow as tf
import matplotlib.pyplot as plt
from pyts.image import RecurrencePlot
from sklearn.model_selection import train_test_split
from tensorflow.keras import optimizers, layers, models, utils
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 1.0
session = tf.compat.v1.Session(config=config)

In [14]:
a12 = pd.read_csv("./Dataset/as/ones/a12.csv").iloc[:4864, 1:9].dropna().to_numpy()
a13 = pd.read_csv("./Dataset/as/ones/a13.csv").iloc[:4864, 1:9].dropna().to_numpy()
a11 = pd.read_csv("./Dataset/as/ones/Letter_A.csv").iloc[:4864, 1:9].dropna().to_numpy()
a21 = pd.read_csv("./Dataset/as/twos/a21.csv").iloc[:4864, 1:9].dropna().to_numpy()
a4 = pd.read_csv("./Dataset/as/fours/a4.csv").iloc[:4864, 1:9].dropna().to_numpy()

as_1 = pd.read_csv("./Dataset/dataset/as_1.csv").dropna().iloc[:2304, 1:9].to_numpy()

ak_1 = pd.read_csv("./Dataset/A11_K.csv").dropna().iloc[0:12032, 1:9].to_numpy()
ak_2 = pd.read_csv("./Dataset/A12_K.csv").dropna().iloc[0:12032, 1:9].to_numpy()

In [15]:
b12 = pd.read_csv("./Dataset/bs/ones/b12.csv").dropna().iloc[:4864, 1:9].to_numpy()
b13 = pd.read_csv("./Dataset/bs/ones/b13.csv").dropna().iloc[:4864, 1:9].to_numpy()
b11 = pd.read_csv("./Dataset/bs/ones/Letter_B.csv").dropna().iloc[:4864, 1:9].to_numpy()
b31 = pd.read_csv("./Dataset/bs/threes/b31.csv").dropna().iloc[:4864, 1:9].to_numpy()
b4 = pd.read_csv("./Dataset/bs/fours/b4.csv").dropna().iloc[:4864, 1:9].to_numpy()

bs_1 = pd.read_csv("./Dataset/dataset/bs_1.csv").dropna().iloc[:2304, 1:9].to_numpy()

bk_1 = pd.read_csv("./Dataset/B11_K.csv").dropna().iloc[:12032, 1:9].to_numpy()
bk_2 = pd.read_csv("./Dataset/B12_K.csv").dropna().iloc[:12032, 1:9].to_numpy()



In [16]:
base1s = pd.read_csv("./Dataset/baselines/Baseline_EEG.csv").dropna().iloc[:, 1:9]
base2s = pd.read_csv("./Dataset/baselines/rest2.csv").dropna().iloc[:, 1:9]
base3s = pd.read_csv("./Dataset/baselines/rest3.csv").dropna().iloc[:, 1:9]
base4s = pd.read_csv("./Dataset/baselines/base4.csv").dropna().iloc[:, 1:9]


base1s_1 = pd.read_csv("./Dataset/dataset/rest_s1.csv").dropna().iloc[:, 1:9]

base1k_1 = pd.read_csv("./Dataset/rest_eeg_K.csv").dropna().iloc[:, 1:9]

In [17]:
mean_1 = base1s.describe().iloc[1].to_numpy().reshape((1, 8))
std_1 = base1s.describe().iloc[2].to_numpy().reshape((1, 8))

mean_2 = base2s.describe().iloc[1].to_numpy().reshape((1, 8))
std_2 = base2s.describe().iloc[2].to_numpy().reshape((1, 8))

mean_3 = base3s.describe().iloc[1].to_numpy().reshape((1, 8))
std_3 = base3s.describe().iloc[2].to_numpy().reshape((1, 8))

mean_4 = base4s.describe().iloc[1].to_numpy().reshape((1, 8))
std_4 = base4s.describe().iloc[2].to_numpy().reshape((1, 8))

mean1_s = base1s_1.describe().iloc[1].to_numpy().reshape((1, 8))
std1_s = base1s_1.describe().iloc[2].to_numpy().reshape((1, 8))

mean1_k = base1k_1.describe().iloc[1].to_numpy().reshape((1, 8))
std1_k = base1k_1.describe().iloc[2].to_numpy().reshape((1, 8))

In [18]:
a12_new = np.zeros((19, 256, 8))
a13_new = np.zeros((19, 256, 8))
a11_new = np.zeros((19, 256, 8))
a21_new = np.zeros((19, 256, 8))
a4_new = np.zeros((19, 256, 8))
as_new = np.zeros((9, 256, 8))
ak1_new = np.zeros((46, 256, 8))
ak2_new = np.zeros((46, 256, 8))

b12_new = np.zeros((19, 256, 8))
b13_new = np.zeros((19, 256, 8))
b11_new = np.zeros((19, 256, 8))
b31_new = np.zeros((19, 256, 8))
b4_new = np.zeros((19, 256, 8))
bs_new = np.zeros((9, 256, 8))
bk1_new = np.zeros((46, 256, 8))
bk2_new = np.zeros((46, 256, 8))

for i in range(19):
    for j in range(256):
        for k in range(8):
            a12_new[i, j, k] = (a12[i+j, k] - mean_1[0, k])/std_1[0, k]
            a13_new[i, j, k] = (a13[i+j, k] - mean_1[0, k])/std_1[0, k]
            a11_new[i, j, k] = (a11[i+j, k] - mean_1[0, k])/std_1[0, k]
            a21_new[i, j, k] = (a21[i+j, k] - mean_2[0, k])/std_2[0, k]
            a4_new[i, j, k] = (a4[i+j, k] - mean_4[0, k])/std_4[0, k]



            b12_new[i, j, k] = (b12[i+j, k] - mean_1[0, k])/std_1[0, k]
            b13_new[i, j, k] = (b13[i+j, k] - mean_1[0, k])/std_1[0, k]
            b11_new[i, j, k] = (b11[i+j, k] - mean_1[0, k])/std_1[0, k]
            b31_new[i, j, k] = (b31[i+j, k] - mean_3[0, k])/std_3[0, k]
            b4_new[i, j, k] = (b4[i+j, k] - mean_4[0, k])/std_4[0, k]

for i in range(9):
    for j in range(256):
        for k in range(8):
            as_new[i, j, k] = (as_1[i+j, k] - mean1_s[0, k])/std1_s[0, k]
            bs_new[i, j, k] = (bs_1[i+j, k] - mean1_s[0, k])/std1_s[0, k]


for i in range(46):
    for j in range(256):
        for k in range(8):
            ak1_new[i, j, k] = (ak_1[i+j, k] - mean1_k[0, k])/std1_k[0, k]
            ak2_new[i, j, k] = (ak_2[i+j, k] - mean1_k[0, k])/std1_k[0, k]

            bk1_new[i, j, k] = (bk_1[i+j, k] - mean1_k[0, k])/std1_k[0, k]
            bk2_new[i, j, k] = (bk_2[i+j, k] - mean1_k[0, k])/std1_k[0, k]

In [19]:
all_a_s = np.concatenate((a12_new, a13_new, a11_new, a21_new, a4_new, as_new, ak1_new, ak2_new), axis = 0)
all_b_s = np.concatenate((b12_new, b13_new, b11_new, b31_new, b4_new, bs_new, bk1_new, bk2_new), axis = 0)

all_a_s = all_a_s.reshape((196, 256, 8))
all_b_s = all_b_s.reshape((196, 256, 8))

## EMD

In [20]:
def emd_signals(input_data):
    imf_count = 7
    emd = EMD()
    #input data is of the size: (196,256,8)
    #For EMD, the input data will be of the form: (trails, time_points, channels)
    #We get the data info first:
    trails, time_points, channels = input_data.shape
    imfs = np.zeros((trails, channels, imf_count, time_points))
    # the output data will be of the type: (trials, channels, num_imfs, time points)
    
    for i in range(trails):
        for j in range(channels):
            signal = input_data[i,:,j]
            imf_val = emd(signal)
            
            #For safety reasons, we will pad with zeros incase the number of IMFs obtained are lesser than what we need
            
            if len(imf_val) < imf_count:
                imf_padded = np.pad(imf_val, ((0, imf_count - len(imf_val)), (0,0)), mode = 'constant')
            else:
                imf_padded = imf_val[:imf_count]
            
            imfs[i,j] = imf_padded
    return imfs

In [ ]:
a_s1 = emd_signals(np.array(all_a_s))
b_s1 = emd_signals(np.array(all_b_s))

In [ ]:
def emd_features(imfs):
    #We take the imf data for all the signals and get the statisitcal features which are:
    # mean, variance, energy and entropy.
    # these may seem random but all of them are generally used along with skewness and kurtosis for effective feature extraction in non stationary signals
    #The input data is of the form: (trials, channels, num_imfs, time points)
    
    trials, channels, num_imfs, time_points = imfs.shape
    features = []
    
    for i in range(trials):
        trail_feature = []
        for j in range(channels):
            for k in range(num_imfs):
                imf = imfs[i,j,k]
                
                energy = np.sum(imf**2)
                entropy = -np.sum(imf * np.log2(np.abs(imf) + 1e-12))
                mean = np.mean(imf)
                variance = np.var(imf)
                
                trail_feature.extend([energy,entropy,mean,variance])
        
        features.append(trail_feature)
        
    return np.array(features)
                

In [ ]:
features_a1 = emd_features(a_s1)
features_b1 = emd_features(b_s1)

In [ ]:
features_a = features_a1.reshape((196,32,7))
features_b = features_b1.reshape((196,32,7))

# Recurrence Plot

In [21]:
def reccurence_plot(input_mats):
    out_mats = []
    for i in range(input_mats.shape[0]):
        transformer = RecurrencePlot()
        X_new = transformer.transform(input_mats[i, :, :])
        out_mats.append(X_new)
    return np.array(out_mats)

In [23]:
image_a_s = reccurence_plot(np.array(features_a))
image_b_s = reccurence_plot(np.array(features_b))
X = [image_a_s, image_b_s]
X = np.array(X).reshape(392, 1, 32, 7, 7)

In [24]:
num_of_output = 2
y = np.array([[0]*196, [1]*196]).reshape((392,))
y = utils.to_categorical(y, num_of_output)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=24)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=0.8, test_size=0.2, random_state=16)

print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)

print(X_val.shape)
print(y_val.shape)

In [27]:
with open("X_train_DL", 'wb') as fp:
    pickle.dump(X_train, fp)

with open("X_test_DL", 'wb') as fp:
    pickle.dump(X_test, fp)

with open("X_val_DL", 'wb') as fp:
    pickle.dump(X_val, fp)

with open("y_train_DL", 'wb') as fp:
    pickle.dump(y_train, fp)

with open("y_test_DL", 'wb') as fp:
    pickle.dump(y_test, fp)

with open("y_val_DL", 'wb') as fp:
    pickle.dump(y_val, fp)


In [ ]:
# Input shape (make sure it matches your data shape)
input_shape = (1, 32, 7, 7)

# RMSprop optimizer
opt = optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07
)


# Model architecture
input_1 = layers.Input(shape=input_shape, name='input_1')
permuted_input = layers.Permute((1, 3, 4, 2), name='permuted_input')(input_1)

# Conv3D layers
conv3d_1 = layers.Conv3D(filters=16, kernel_size=(8, 5, 5), padding='same', activation='relu', kernel_initializer="glorot_uniform", name='conv3d_1')(permuted_input)
batch_1 = layers.BatchNormalization(name='batch_1')(conv3d_1)
maxpool_1 = layers.MaxPool3D(pool_size=(2, 2, 2), strides=(1, 1, 1), padding='same', name='maxpool_1')(batch_1)

conv3d_2 = layers.Conv3D(filters=8, kernel_size=(4, 4, 4), padding='same', activation='relu', kernel_initializer="glorot_uniform", name='conv3d_2')(maxpool_1)
batch_2 = layers.BatchNormalization(name='batch_2')(conv3d_2)
maxpool_2 = layers.MaxPool3D(pool_size=(2, 2, 2), strides=(1, 1, 1), padding='same', name='maxpool_2')(batch_2)

conv3d_3 = layers.Conv3D(filters=8, kernel_size=(4, 3, 3), padding='same', activation='relu', kernel_initializer="glorot_uniform", name='conv3d_3')(maxpool_2)
batch_3 = layers.BatchNormalization(name='batch_3')(conv3d_3)
maxpool_3 = layers.MaxPool3D(pool_size=(2, 2, 2), strides=(1, 1, 1), padding='same', name='maxpool_3')(batch_3)

# Flatten layer
flatten = layers.Flatten()(maxpool_3)

# Dense layers
dense_1 = layers.Dense(units=16, activation='relu')(flatten)
dense_2 = layers.Dense(units=16, activation='relu')(dense_1)

# Output layer (change num_of_output as per your problem)
output_layer = layers.Dense(num_of_output, activation="softmax")(dense_2)

# Model compilation
eeg_model = models.Model(inputs=input_1, outputs=output_layer)

# Compile the model with loss and optimizer
eeg_model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])  # Change loss if binary classification

# Model summary
eeg_model.summary()


In [ ]:
# EarlyStopping callback to stop training if validation loss does not improve
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Reduce learning rate when the validation loss plateaus
lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=1e-6)

# Train the model with training and validation data
history = eeg_model.fit(
    x=X_train, y=y_train,
    batch_size=8,
    epochs=20,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, lr_scheduler]
)


In [ ]:
acc = history.history['accuracy']
loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']

plt.subplot(1, 2, 1)
plt.plot(acc)
plt.plot(val_acc)
plt.title("Accuracy vs Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Loss Value")

plt.subplot(1, 2, 2)
plt.plot(loss)
plt.plot(val_loss)
plt.title("Loss vs Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss Value")
plt.show()

In [ ]:
y_pred = eeg_model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)

# Classification Report
print(classification_report(y_test, y_pred)) 

conf_mat = confusion_matrix(y_test, y_pred)
cm_display = ConfusionMatrixDisplay(confusion_matrix = conf_mat, display_labels = ["A", "B"])
cm_display.plot()
plt.show()

In [ ]:
eeg_model.save("eeg_model_100_all.h5")